In [1]:
from lbmpy.session import *
from lbmpy.phasefield_allen_cahn.kernel_equations import *
from pystencils import fields
from pystencils import Field

from lbmpy.creationfunctions import create_lb_method

In [2]:
def apply(field_access: Field.Access, stencil, weights):
    f = field_access
    return sum(f.get_shifted(*offset) * weight for offset, weight in zip(stencil, weights))

# test chemical potencial

In [3]:
stencil = LBStencil(Stencil.D2Q9)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

beta = 0
kappa = 1

a = chemical_potential_symbolic(C, stencil, beta, kappa)

expected_result = sp.Array([20, -4, -4, -4, -4, -1, -1, -1, -1]) / 6
b = apply(C.center, stencil, expected_result)

assert a == b

In [4]:
stencil = LBStencil(Stencil.D3Q15)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

beta = 0
kappa = 1

a = chemical_potential_symbolic(C, stencil, beta, kappa)

expected_result = sp.Array([256, -28, -28, -28, -28, -28, -28, -11, -11, -11, -11, -11, -11, -11, -11]) / 72
b = apply(C.center, stencil, expected_result)

assert a == b

In [5]:
stencil = LBStencil(Stencil.D3Q19)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

beta = 0
kappa = 1

a = chemical_potential_symbolic(C, stencil, beta, kappa)

expected_result = sp.Array([24, -2, -2, -2, -2, -2, -2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]) / 6
b = apply(C.center, stencil, expected_result)

assert a == b

In [6]:
stencil = LBStencil(Stencil.D3Q27)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

beta = 0
kappa = 1

a = chemical_potential_symbolic(C, stencil, beta, kappa)

expected_result = sp.Array([152,
                            -16, -16, -16, -16, -16, -16,
                            -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4,
                            -1, -1, -1, -1, -1, -1, -1, -1]) / 36
b = apply(C.center, stencil, expected_result)

assert a == b

# test isotropic gradient

In [7]:
stencil = LBStencil(Stencil.D2Q9)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

a = isotropic_gradient_symbolic(C, stencil)

expected_result = sp.Array([-1, -4, -1, 1, 4, 1]) / 12
expected_grad_stencil = ((-1,-1), (-1,0), (-1,1), (1,-1), (1,0), (1,1))
b = apply(C.center, expected_grad_stencil, expected_result)

assert a[0] == b

In [8]:
stencil = LBStencil(Stencil.D3Q15)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

a = isotropic_gradient_symbolic(C, stencil)

expected_result = sp.Array([-1, -1, -8, -1, -1, 1, 1, 8, 1, 1]) / 24
expected_grad_stencil = ((-1,-1,-1), (-1,-1,1), (-1,0,0), (-1,1,-1), (-1,1,1), (1,-1,-1), (1,-1,1), (1,0,0), (1,1,-1), (1,1,1))
b = apply(C.center, expected_grad_stencil, expected_result)

assert a[0] == b

In [9]:
stencil = LBStencil(Stencil.D3Q19)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

a = isotropic_gradient_symbolic(C, stencil)

expected_result = sp.Array([-1, -1, -2, -1, -1, 1, 1, 2, 1, 1]) / 12
expected_grad_stencil = ((-1,-1,0), (-1,0,-1), (-1,0,0), (-1,0,1), (-1,1,0), (1,-1,0), (1,0,-1), (1,0,0), (1,0,1), (1,1,0))
b = apply(C.center, expected_grad_stencil, expected_result)

assert a[0] == b

In [10]:
stencil = LBStencil(Stencil.D3Q27)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')

a = isotropic_gradient_symbolic(C, stencil)

expected_result = sp.Array([-1, -4, -1, -4, -16, -4, -1, -4, -1, 1, 4, 1, 4, 16, 4, 1, 4, 1]) / 72
expected_grad_stencil = ((-1,-1,-1), (-1,-1,0), (-1,-1,1), (-1,0,-1), (-1,0,0), (-1,0,1), (-1,1,-1), (-1,1,0), (-1,1,1),
                         (1,-1,-1), (1,-1,0), (1,-1,1), (1,0,-1), (1,0,0), (1,0,1), (1,1,-1), (1,1,0), (1,1,1))
b = apply(C.center, expected_grad_stencil, expected_result)

assert a[0] == b

In [11]:
ng = normalized_isotropic_gradient_symbolic(C, stencil)

tmp = (sum(map(lambda x: x * x, a)) + 1.e-32) ** 0.5 

assert ng[0] == a[0] / tmp

# test hydrodynamic force

In [12]:
stencil = LBStencil(Stencil.D3Q27)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')
g = fields("lb_velocity_field(" + str(len(stencil)) + "): [" + str(dimensions) + "D]", layout='fzyx')

tau = 0.53

lbm_config = LBMConfig(stencil=stencil, method=Method.MRT, relaxation_rates=[1/tau, 1, 1, 1, 1, 1])
lb_method = create_lb_method(lbm_config=lbm_config)

In [13]:
a = hydrodynamic_force(g, C, lb_method, tau, 1, 0.1, 1, 0, [0, 0, 0] , fd_stencil=None)
b = hydrodynamic_force(g, C, lb_method, tau, 1, 0.1, 1, 0, [0, 0, 0] , fd_stencil=LBStencil(Stencil.D3Q27))
c = hydrodynamic_force(g, C, lb_method, tau, 1, 0.1, 1, 0, [0, 0, 0] , fd_stencil=LBStencil(Stencil.D3Q19))
d = hydrodynamic_force(g, C, lb_method, tau, 1, 0.1, 1, 0, [0, 0, 0] , fd_stencil=LBStencil(Stencil.D3Q15))

In [14]:
pf = pressure_force(C, stencil, 1, 0.1)
vf = viscous_force(g, C, lb_method, tau, 1, 0.1)
sf = surface_tension_force(C, stencil, 0, 1)

assert sp.simplify(pf[0] + vf[0] + sf[0] - b[0]) == 0
assert sp.simplify(pf[1] + vf[1] + sf[1] - b[1]) == 0
assert sp.simplify(pf[2] + vf[2] + sf[2] - b[2]) == 0

In [15]:
stencil = LBStencil(Stencil.D2Q9)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')
g = fields("lb_velocity_field(" + str(len(stencil)) + "): [" + str(dimensions) + "D]", layout='fzyx')

tau = 0.53

lbm_config = LBMConfig(stencil=stencil, method=Method.MRT, relaxation_rates=[1/tau, 1, 1, 1, 1, 1])
lb_method = create_lb_method(lbm_config=lbm_config)

a = hydrodynamic_force(g, C, lb_method, tau, 1, 0.1, 1, 0, [0, 0, 0] , fd_stencil=None)
b = hydrodynamic_force(g, C, lb_method, tau, 1, 0.1, 1, 0, [0, 0, 0] , fd_stencil=stencil)

In [16]:
stencil = LBStencil(Stencil.D3Q27)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')
u = fields("vel_field(" + str(dimensions) + "): [" + str(dimensions) + "D]", layout='fzyx')
h = fields("lb_velocity_field(" + str(len(stencil)) + "): [" + str(dimensions) + "D]", layout='fzyx')

tau = 0.53

lbm_config = LBMConfig(stencil=stencil, method=Method.SRT)
lb_method = create_lb_method(lbm_config=lbm_config)

a = initializer_kernel_phase_field_lb(h, C, u, lb_method, 5, fd_stencil=None)
b = initializer_kernel_phase_field_lb(h, C, u, lb_method, 5, fd_stencil=LBStencil(Stencil.D3Q27))
c = initializer_kernel_phase_field_lb(h, C, u, lb_method, 5, fd_stencil=LBStencil(Stencil.D3Q19))
d = initializer_kernel_phase_field_lb(h, C, u, lb_method, 5, fd_stencil=LBStencil(Stencil.D3Q15))

In [17]:
stencil = LBStencil(Stencil.D2Q9)
dimensions = len(stencil[0])
C = fields("phase_field: [" + str(dimensions) + "D]", layout='fzyx')
u = fields("vel_field(" + str(dimensions) + "): [" + str(dimensions) + "D]", layout='fzyx')
h = fields("lb_velocity_field(" + str(len(stencil)) + "): [" + str(dimensions) + "D]", layout='fzyx')

tau = 0.53

lbm_config = LBMConfig(stencil=stencil, method=Method.SRT)
lb_method = create_lb_method(lbm_config=lbm_config)

a = initializer_kernel_phase_field_lb(h, C, u, lb_method, 5, fd_stencil=None)
b = initializer_kernel_phase_field_lb(h, C, u, lb_method, 5, fd_stencil=stencil)